In [1]:
import numpy as np
from scipy.interpolate import griddata
import netCDF4
from dask.distributed import Client
import dask.array as da
import dask

In [2]:
#localhost:8787/status
client=Client(n_workers=16,
               threads_per_worker=2,
               memory_target_fraction=0.95,
               memory_limit='1GB')

2025-01-05 05:04:10,293 - distributed.semaphore - WARNING - Tried to release Lock or Semaphore but it was already released: name='C:\\Users\\Kai Wen Lee\\Documents\\Journal-KW-MASTER\\NOB\\JupyterNotebooks\\Mesh\\corsetofine\\flow0000.nc', lease_id='40323e6fb1c94599b724d89c8be732f6'. This can happen if the Lock or Semaphore timed out before.
2025-01-05 05:04:10,341 - distributed.semaphore - WARNING - Tried to release Lock or Semaphore but it was already released: name='C:\\Users\\Kai Wen Lee\\Documents\\Journal-KW-MASTER\\NOB\\JupyterNotebooks\\Mesh\\corsetofine\\flow0000.nc', lease_id='381a071f669440b2aacda85d46058b61'. This can happen if the Lock or Semaphore timed out before.
2025-01-05 05:20:53,072 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "C:\Users\Kai Wen Lee\AppData\Local

In [3]:
g_old = netCDF4.Dataset('geometry_old.nc', 'r')
zu_coarse = da.from_array(g_old['z_uz'])
zp_coarse = da.from_array(g_old['z_p'])
dz_coarse = da.from_array(g_old['delta_z_p'])
rp_coarse = da.from_array(g_old['r_p'])
rw_coarse = da.from_array(g_old['r_ur'])
dr_coarse = da.from_array(g_old['delta_r_p'])
dphi_coarse = da.from_array(g_old['delta_phi'])

dimz_coarse = len(g_old.dimensions['dim_z'])
dimphi_coarse = len(g_old.dimensions['dim_phi'])
dimr_coarse = len(g_old.dimensions['dim_r'])

height_coarse = da.max(zp_coarse)
phi_coarse = da.arange(dimphi_coarse) * dphi_coarse - np.pi - dphi_coarse - dphi_coarse / 2.0

In [4]:
g_new = netCDF4.Dataset('geometry.nc', 'r')
zu_fine = da.from_array(g_new['z_uz'])
zp_fine = da.from_array(g_new['z_p'])
dz_fine = da.from_array(g_new['delta_z_p'])
rp_fine = da.from_array(g_new['r_p'])
rw_fine = da.from_array(g_new['r_ur'])
dr_fine = da.from_array(g_new['delta_r_p'])
dphi_fine = da.from_array(g_new['delta_phi'])

dimz_fine = len(g_new.dimensions['dim_z'])
dimphi_fine = len(g_new.dimensions['dim_phi'])
dimr_fine = len(g_new.dimensions['dim_r'])

height_fine = da.max(zp_fine)
phi_fine = da.arange(dimphi_fine) * dphi_fine - np.pi - dphi_fine - dphi_fine / 2.0

In [5]:
file='flow0037.nc'
# Read data from coarse mesh
f_old = netCDF4.Dataset(file, 'r')
step = f_old.variables['timestep'][:]
time = f_old.variables['time'][:]
gamma = f_old.variables['gamma'][:]
Ra = f_old.variables['Ra'][:]
Pr = f_old.variables['Pr'][:]
Ro = f_old.variables['Ro'][:]

temp_coarse = da.from_array(f_old['temp'])
uz_coarse = da.from_array(f_old['uz'])
uphi_coarse = da.from_array(f_old['uphi'])
ur_coarse = da.from_array(f_old['ur'])

In [6]:
Z_coarse, R_coarse, Phi_coarse = da.meshgrid(zu_coarse,rp_coarse,phi_coarse)
Z_fine, R_fine, Phi_fine = da.meshgrid(zu_fine,rp_fine,phi_fine)

print(rp_coarse)
print(rp_fine)
print(Phi_fine.shape)
print(temp_coarse)

dask.array<array, shape=(474,), dtype=float64, chunksize=(474,), chunktype=numpy.MaskedArray>
dask.array<array, shape=(1538,), dtype=float64, chunksize=(1538,), chunktype=numpy.MaskedArray>
(1538, 1538, 1028)
dask.array<array, shape=(386, 474, 132), dtype=float64, chunksize=(356, 356, 132), chunktype=numpy.MaskedArray>


In [7]:
def spline(X, Y, T, sigma = 1.0):
    n = min(len(X), len(Y))
    if n <= 2:
        print('X and Y must be arrays of 3 or more elements.')
    if sigma != 1.0:
        sigma = min(sigma, 0.001)
    yp = np.zeros(2*n)
    delx1 = X[1]-X[0]
    dx1 = (Y[1]-Y[0])/delx1
    nm1 = n-1
    nmp = n+1
    delx2 = X[2]-X[1]
    delx12 = X[2]-X[0]
    c1 = -(delx12+delx1)/(delx12*delx1)
    c2 = delx12/(delx1*delx2)
    c3 = -delx1/(delx12*delx2)
    slpp1 = c1*Y[0]+c2*Y[1]+c3*Y[2]
    deln = X[nm1]-X[nm1-1]
    delnm1 = X[nm1-1]-X[nm1-2]
    delnn = X[nm1]-X[nm1-2]
    c1 = (delnn+deln)/(delnn*deln)
    c2 = -delnn/(deln*delnm1)
    c3 = deln/(delnn*delnm1)
    slppn = c3*Y[nm1-2]+c2*Y[nm1-1]+c1*Y[nm1]
    sigmap = sigma*nm1/(X[nm1]-X[0])
    dels = sigmap*delx1
    exps = np.exp(dels)
    sinhs = 0.5*(exps-1/exps)
    sinhin = 1/(delx1*sinhs)
    diag1 = sinhin*(dels*0.5*(exps+1/exps)-sinhs)
    diagin = 1/diag1
    yp[0] = diagin*(dx1-slpp1)
    spdiag = sinhin*(sinhs-dels)
    yp[n] = diagin*spdiag
    delx2 = X[1:]-X[:-1]
    dx2 = (Y[1:]-Y[:-1])/delx2
    dels = sigmap*delx2
    exps = np.exp(dels)
    sinhs = 0.5*(exps-1/exps)
    sinhin = 1/(delx2*sinhs)
    diag2 = sinhin*(dels*(0.5*(exps+1/exps))-sinhs)
    diag2 = np.concatenate([np.array([0]), diag2[:-1]+diag2[1:]])
    dx2nm1 = dx2[nm1-1]
    dx2 = np.concatenate([np.array([0]), dx2[1:]-dx2[:-1]])
    spdiag = sinhin*(sinhs-dels)
    for i in range(1, nm1):
        diagin = 1/(diag2[i]-spdiag[i-1]*yp[i+n-1])
        yp[i] = diagin*(dx2[i]-spdiag[i-1]*yp[i-1])
        yp[i+n] = diagin*spdiag[i]
    diagin = 1/(diag1-spdiag[nm1-1]*yp[n+nm1-1])
    yp[nm1] = diagin*(slppn-dx2nm1-spdiag[nm1-1]*yp[nm1-1])
    for i in range(n-2, -1, -1):
        yp[i] = yp[i]-yp[i+n]*yp[i+1]
    m = len(T)
    subs = np.repeat(nm1, m)
    s = X[nm1]-X[0]
    sigmap = sigma*nm1/s
    j = 0
    for i in range(1, nm1+1):
        while T[j] < X[i]:
            subs[j] = i
            j += 1
            if j == m:
                break
        if j == m:
            break
    subs1 = subs-1
    del1 = T-X[subs1]
    del2 = X[subs]-T
    dels = X[subs]-X[subs1]
    exps1 = np.exp(sigmap*del1)
    sinhd1 = 0.5*(exps1-1/exps1)
    exps = np.exp(sigmap*del2)
    sinhd2 = 0.5*(exps-1/exps)
    exps = exps1*exps
    sinhs = 0.5*(exps-1/exps)
    spl = (yp[subs]*sinhd1+yp[subs1]*sinhd2)/sinhs+((Y[subs]-yp[subs])*del1+(Y[subs1]-yp[subs1])*del2)/dels
    if m == 1:
        return spl[0]
    else:
        return spl



In [ ]:
test = da.zeros([12, 12, 12])
print(test.shape)
test[:, 0, 0] = spline(
        da.array([1, 2, 3, 4, 5, 6]), 
        da.array([2, 2, 2, 4, 4, 4]), 
        da.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
    )
test[0, :, 0] = spline(
        da.array([1, 2, 3, 4, 5, 6]), 
        da.array([5, 5, 5, 4, 4, 4]), 
        da.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
    )
test[0, 0, :] = spline(
    da.array([1, 2, 3, 4, 5, 6]), 
    da.array([100, 200, 200, 400, 400, 400]), 
    da.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
)
print(test)
#print(test.compute())

In [ ]:
testc=da.linspace(0,1,num=10)
testf=da.linspace(0,1,num=100)
testrng = da.random.default_rng()
testvals = testrng.standard_normal(10)

@dask.delayed
def testfunc(cc,vals,ff):
    return spline(cc,vals,ff)

@dask.delayed
def output(arr1,arr2,arr3):
    test=da.zeros([100,100,100])
    test[:,0,0]=arr1
    test[0,:,0]=arr2
    test[0,0,:]=arr3
    return test
    
task1 =testfunc(testc,testvals,testf)
task2 =testfunc(testc,testvals,testf)
task3 =testfunc(testc,testvals,testf)
combineheuhue = output(task1,task2,task3)

# Combine the tasks and compute them in parallel
results = combineheuhue.compute()

# Print the shapes of the computed arrays
print(results.shape)

In [8]:
def convert_to_array(maskedarr):
    computed_array=maskedarr.compute()
    tempndarr = computed_array.data
    return da.array(tempndarr)

temp_coarse=convert_to_array(temp_coarse)
rp_coarse=convert_to_array(rp_coarse)
rp_fine=convert_to_array(rp_fine)

phi_coarse=convert_to_array(phi_coarse)
phi_fine=convert_to_array(phi_fine)

zu_coarse=convert_to_array(zu_coarse)
zu_fine=convert_to_array(zu_fine)

temp_coarse=convert_to_array(temp_coarse)
rp_coarse=convert_to_array(rp_coarse)
rp_fine=convert_to_array(rp_fine)

phi_coarse=convert_to_array(phi_coarse)
phi_fine=convert_to_array(phi_fine)

zu_coarse=convert_to_array(zu_coarse)
zu_fine=convert_to_array(zu_fine)

temp_coarse=convert_to_array(temp_coarse)
rp_coarse=convert_to_array(rp_coarse)
rp_fine=convert_to_array(rp_fine)

phi_coarse=convert_to_array(phi_coarse)
phi_fine=convert_to_array(phi_fine)

zu_coarse=convert_to_array(zu_coarse)
zu_fine=convert_to_array(zu_fine)

uz_coarse=convert_to_array(uz_coarse)
ur_coarse=convert_to_array(ur_coarse)
uphi_coarse=convert_to_array(uphi_coarse)

C:\Users\Kai Wen Lee\AppData\Local\Programs\Python\Python313\Lib\site-packages\distributed\client.py:3371: UserWarning: Sending large graph of size 184.26 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [9]:
@dask.delayed
def splinewrapper(finearr,data,coarsearr):
    return spline(finearr,data,coarsearr)

@dask.delayed
def output(arrz,arrr,arrphi):
    arrout=da.zeros([dimz_fine,dimr_fine,dimphi_fine])
    arrout[0,:,0]=arrz
    arrout[0,0,:]=arrr
    arrout[:,0,0]=arrphi
    return arrout


tempr =splinewrapper(rp_coarse,temp_coarse[0,:,0],rp_fine)
tempphi =splinewrapper(phi_coarse,temp_coarse[0,0,:],phi_fine)
tempz =splinewrapper(zu_coarse,temp_coarse[:,0,0],zu_fine)
tempf = output(tempr,tempphi,tempz)

# Combine the tasks and compute them in parallel
temp_fine = tempf.compute()

# Print the shapes of the computed arrays
print(temp_fine.shape)

C:\Users\Kai Wen Lee\AppData\Local\Programs\Python\Python313\Lib\site-packages\distributed\client.py:3371: UserWarning: Sending large graph of size 180.74 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


(1538, 1538, 1028)


In [10]:
uzr =splinewrapper(rp_coarse,uz_coarse[0,:,0],rp_fine)
uzphi =splinewrapper(phi_coarse,uz_coarse[0,0,:],phi_fine)
uzz =splinewrapper(zu_coarse,uz_coarse[:,0,0],zu_fine)
uzf = output(uzr,uzphi,uzz)

# Combine the tasks and compute them in parallel
uz_fine = uzf.compute()

# Print the shapes of the computed arrays
print(uz_fine.shape)

(1538, 1538, 1028)


In [11]:
urr =splinewrapper(rp_coarse,uz_coarse[0,:,0],rp_fine)
urphi =splinewrapper(phi_coarse,uz_coarse[0,0,:],phi_fine)
urz =splinewrapper(zu_coarse,uz_coarse[:,0,0],zu_fine)
urf = output(uzr,uzphi,uzz)

# Combine the tasks and compute them in parallel
ur_fine = urf.compute()

# Print the shapes of the computed arrays
print(ur_fine.shape)

(1538, 1538, 1028)


In [12]:
uphir =splinewrapper(rp_coarse,uphi_coarse[0,:,0],rp_fine)
uphiphi =splinewrapper(phi_coarse,uphi_coarse[0,0,:],phi_fine)
uphiz =splinewrapper(zu_coarse,uphi_coarse[:,0,0],zu_fine)
uphif = output(uzr,uzphi,uzz)

# Combine the tasks and compute them in parallel
uphi_fine = urf.compute()

# Print the shapes of the computed arrays
print(uphi_fine.shape)

(1538, 1538, 1028)


In [13]:
#Write everything, need to use Xarrays for this
import xarray as xr
flow_coarse = xr.open_dataset("flow0037.nc", engine="netcdf4",format='netCDF4')
flow_fine = xr.Dataset().chunk(chunks={"auto"})
flow_fine['Ra']=flow_coarse.Ra
flow_fine['Pr']=flow_coarse.Pr
flow_fine['Ro']=flow_coarse.Ro
flow_fine['gamma']=flow_coarse.gamma
flow_fine['time']=flow_coarse.time
flow_fine['timestep']=flow_coarse.timestep
flow_fine['temp'] = xr.DataArray(temp_fine)
flow_fine['uz'] = xr.DataArray(uz_fine)
flow_fine['uphi'] = xr.DataArray(uphi_fine)
flow_fine['ur'] = xr.DataArray(ur_fine) 
print('Flow file summary:')
print(flow_fine)
print('Comparison with course file:')
print(flow_coarse)

Flow file summary:
<xarray.Dataset> Size: 78GB
Dimensions:   (dim_0: 1538, dim_1: 1538, dim_2: 1028)
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    Ra        float64 8B ...
    Pr        float64 8B ...
    Ro        float64 8B ...
    gamma     float64 8B ...
    time      float64 8B ...
    timestep  int32 4B ...
    temp      (dim_0, dim_1, dim_2) float64 19GB dask.array<chunksize=(255, 255, 255), meta=np.ndarray>
    uz        (dim_0, dim_1, dim_2) float64 19GB dask.array<chunksize=(255, 255, 255), meta=np.ndarray>
    uphi      (dim_0, dim_1, dim_2) float64 19GB dask.array<chunksize=(255, 255, 255), meta=np.ndarray>
    ur        (dim_0, dim_1, dim_2) float64 19GB dask.array<chunksize=(255, 255, 255), meta=np.ndarray>
Comparison with course file:
<xarray.Dataset> Size: 773MB
Dimensions:   (dim_z: 386, dim_r: 474, dim_phi: 132)
Dimensions without coordinates: dim_z, dim_r, dim_phi
Data variables:
    Ra        float64 8B ...
    Pr        float64 8B ...
    

In [18]:
print('writing')
encoding = {"temp": {"zlib": True, "complevel": 9},
            "ur": {"zlib": True, "complevel": 9},
            "uphi": {"zlib": True, "complevel": 9},
            "uz": {"zlib": True, "complevel": 9}}
flow_fine.to_netcdf("flow0000.nc", format="NETCDF4",engine="netcdf4",mode="w",encoding=encoding)

writing
